In [1]:
# Import plotting libraries
import matplotlib 
from matplotlib import pyplot as plt
matplotlib.rcParams.update({'font.size': 20})
plt.rcParams.update({'font.size': 22})

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

print(keras.__version__)
import numpy as np

print(np.__version__)
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow

%load_ext autoreload
%autoreload 2


Using TensorFlow backend.


2.3.1
1.21.2


In [2]:
seed = 12

In [3]:
data = pd.read_csv('bg_struct.csv')

In [4]:
data.head()

,Unnamed: 0,pretty_formula,bandgap,cell_length_a,cell_lentgh_b,cell_lentgh_c,cell_angle_alpha,cell_angle_beta,cell_angle_gamma
0,0,BrCl,1.8278,4.645947,4.645947,7.810701,90.000000,90.000000,115.948189
1,1,SrC2,1.8008,3.710786,3.710786,3.710785,85.434482,85.434482,85.434493
2,2,Cs2Se,1.8027,5.624158,9.095804,10.998913,90.000000,90.000000,90.000000
3,3,Cs2Se,1.8141,9.136351,5.630188,12.173633,63.874547,90.000000,90.000000
4,4,CsPbBr3,1.8178,8.459612,8.459612,8.459612,120.097716,120.097716,89.830834


In [5]:
data.shape

(16335, 9)

In [6]:
X = data[['cell_length_a', 'cell_lentgh_b', 'cell_lentgh_c', 'cell_angle_alpha', 'cell_angle_beta', 'cell_angle_gamma']].values
Y = data[['bandgap']].values

In [7]:
# create test/train split from the data above and  keep 20% for testing
X_train_pn, X_test_pn, y_train, y_test = train_test_split(X, Y,
                                                         test_size=0.20, 
                                                         random_state=seed)

# train a standard scaler model to apply it to the train set and test set
X_train_scaler = StandardScaler().fit(X_train_pn)
X_train = X_train_scaler.transform(X_train_pn)
X_test = X_train_scaler.transform(X_test_pn)


### Create a neural network model

In [ ]:
def neural_network_model(n1, n2, act1, act2, lr):
    """
    activation: 'relu', 'sigmoid', 'linear', 'tanh', 'selu', 'elu'
    """
    model = Sequential()
    model.add(Dense(n1, input_dim=6, kernel_initializer='normal', activation=act1))
    model.add(Dense(n2, kernel_initializer='normal', activation=act2))
    model.add(Dense(1, kernel_initializer='normal'))
    
    opt = tensorflow.keras.optimizers.Adam(learning_rate=lr)
    
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [12]:
np.random.seed(seed)
# Create the NN framework
estimator = KerasRegressor(build_fn=neural_network_model, 
                           n1=60, n2=50, act1='relu', act2='linear', 
                           epochs=150, batch_size=10000, verbose=0, lr=0.9)
# Fit to training data
history = estimator.fit(X_train, y_train, validation_split=0.30, epochs=150, 
        batch_size=10000, verbose=0)

print("Final MSE for train is %.3e and for validation is %.3e" % 
      (history.history['loss'][-1], history.history['val_loss'][-1]))

Final MSE for train is 3.956e+00 and for validation is 3.969e+00


In [10]:
def neural_network_model(n1, n2, act1, act2, lr):
    """
    activation: 'relu', 'sigmoid', 'linear', 'tanh', 'selu', 'elu'
    """
    model = Sequential()
    model.add(Dense(n1, input_dim=6, kernel_initializer='normal', activation=act1))
    model.add(Dense(n2, kernel_initializer='normal', activation=act2))
    model.add(Dense(1, kernel_initializer='normal'))
    
    opt = tensorflow.keras.optimizers.Adam(learning_rate=lr)
    
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [11]:
np.random.seed(seed)

MSE = []
lr_vals = [0.00001, 0.0001, 0.01, 0.1, 1.0]
for lrate in lr_vals:
    estimator = KerasRegressor(build_fn=neural_network_model,
                               n1=60, n2=50, act1='relu', act2='linear', 
                               epochs=150, batch_size=10000, 
                               verbose=0, lr=lrate)
    history = estimator.fit(X_train, y_train, validation_split=0.30,
                            epochs=150, batch_size=10000, 
                            verbose=0)
    print("Final MSE for train is %.3e and for validation is %.3e" % 
      (history.history['loss'][-1], history.history['val_loss'][-1]))
    MSE.append([history.history['loss'][-1], 
                history.history['val_loss'][-1],
                estimator.model.evaluate(X_test, y_test)])

2022-03-06 23:05:09.290067: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-06 23:05:09.290662: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-06 23:05:09.295322: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-03-06 23:05:09.366173: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-06 23:05:09.367728: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3193885000 Hz


Final MSE for train is 5.928e+00 and for validation is 5.854e+00
103/103 [==============================] - 0s 4ms/step - loss: 5.8832
Final MSE for train is 4.871e+00 and for validation is 4.794e+00
103/103 [==============================] - 0s 2ms/step - loss: 4.8180
Final MSE for train is 1.370e-01 and for validation is 1.411e-01
103/103 [==============================] - 0s 4ms/step - loss: 0.1381
Final MSE for train is 1.372e-01 and for validation is 1.414e-01
103/103 [==============================] - 0s 2ms/step - loss: 0.1389
Final MSE for train is 1.221e+00 and for validation is 4.318e+01
103/103 [==============================] - 0s 2ms/step - loss: 2.5751
